<a href="https://colab.research.google.com/github/Neofly4023/bso-fs/blob/master/bso_fs_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature selection using reinforcement learning

In [0]:
7import pandas as pd
import numpy as np
import os, glob, random, copy, sys, time
from collections import defaultdict
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler


class stream_tee(object):
    def __init__(self, stream1, stream2):
        self.stream1 = stream1
        self.stream2 = stream2
        self.__missing_method_name = None # Hack!
 
    def __getattribute__(self, name):
        return object.__getattribute__(self, name)
 
    def __getattr__(self, name):
        self.__missing_method_name = name # Could also be a property
        return getattr(self, '__methodmissing__')
 
    def __methodmissing__(self, *args, **kwargs):
            # Emit method call to the log copy
            callable2 = getattr(self.stream2, self.__missing_method_name)
            callable2(*args, **kwargs)
 
            # Emit method call to stdout (stream 1)
            callable1 = getattr(self.stream1, self.__missing_method_name)
            return callable1(*args, **kwargs)
      
class Bee :
    def __init__(self,id,problem,locIterations):
        self.id=id
        self.data=problem
        self.solution=[]
        self.fitness= 0.0
        self.reward = 0.0
        self.locIterations=locIterations
        self.action = []
    
    def localSearch(self):
        best=self.fitness
        #done=False
        lista=[j for j, n in enumerate(self.solution) if n == 1]
        indice =lista[0]
        
        for itr in range(self.locIterations):
            while(True):
                pos=-1
                oldFitness=self.fitness
                for i in range(len(self.solution)):
                    
                    if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                        i+=1
                    self.solution[i]= (self.solution[i] + 1) % 2
                    
                    quality = self.data.evaluate(self.solution)
                    if (quality >best):
                        pos = i
                        best=quality
                    self.solution[i]= (self.solution[i]+1) % 2
                    self.fitness = oldFitness 
                if (pos != -1):
                    self.solution[pos]= (self.solution[pos]+1)%2
                    self.fitness = best
                else:
                    break
            for i in range(len(self.solution)):
                oldFitness=self.fitness
                if ((len(lista)==1) and (indice==i) and (i < self.data.nb_attribs-1)):
                    i+=1
                self.solution[i]= (self.solution[i] + 1) % 2
                quality = self.data.evaluate(self.solution)
                if (quality<best):
                    self.solution[i]= (self.solution[i] + 1) % 2
                    self.fitness = oldFitness


    def ql_localSearch(self):
        
        for itr in range(5):
       
            state = self.solution.copy()

            next_state, action = self.data.ql.step(self.data,state)
            acc_state = self.data.evaluate(state)
            acc_new_state = self.data.evaluate(next_state)

            if (acc_state < acc_new_state):
                self.reward = acc_new_state
            elif (acc_state > acc_new_state):
                self.reward = acc_new_state - acc_state
            else :
                if (self.data.nbrUn(state) > self.data.nbrUn(next_state) ):
                    self.reward = 0.5 * acc_new_state
                else :
                    self.reward = -0.5 * acc_new_state

            self.fitness = self.data.ql.get_q_value(self.data,state,action)
            self.data.ql.learn(self.data,state,action,self.reward,next_state)
            self.solution = next_state.copy()
        
       
    def setSolution(self,solution):
        self.solution=solution
        self.fitness=self.data.evaluate(solution)
    
    def Rand(self,num): 
        res = [] 
        res = np.random.choice([0,1],size=(num,),p=[2./10,8./10]).tolist()
  
        return res



class Swarm :
    def __init__(self,problem,flip,maxChance,nbrBees,maxIterations,locIterations):
        self.data=problem
        self.flip=flip
        self.maxChance=maxChance
        self.nbChance=maxChance
        self.nbrBees=nbrBees
        self.maxIterations=maxIterations
        self.locIterations=locIterations
        self.beeList=[]
        self.refSolution = Bee(-1,self.data,self.locIterations)
        self.refSolution.setSolution(self.refSolution.Rand(self.data.nb_attribs))
        self.bestSolution = self.refSolution
        self.tabou=[]

    def searchArea(self):    
        i=0
        h=0
        
        self.beeList=[]
        while((i<self.nbrBees) and (i < self.flip) ) :
            #print ("First method to generate")
            
            solution=self.refSolution.solution
            k=0
            while((self.flip*k+h) < len(solution)):
                solution[self.flip*k +h] = ((solution[self.flip*k+h]+1) % 2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations)
            newBee.solution = copy.deepcopy(solution)
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        h=0
        
        while((i<self.nbrBees) and (i< 2*self.flip )):
            #print("Second method to generate")

            solution=self.refSolution.solution
            k=0
            while((k<int(len(solution)/self.flip)) and (self.flip*k+h < len(solution))):
                solution[int(self.data.nb_attribs/self.flip)*h+k] = ((solution[int(self.data.nb_attribs/self.flip)*h+k]+1)%2)
                k+=1
            newBee=Bee(i,self.data,self.locIterations)
            newBee.solution = copy.deepcopy(solution)
            self.beeList.append(newBee)
            
            i+=1
            h=h+1
        while (i<self.nbrBees):
            #print("Random method to generate")
            solution= self.refSolution.solution
            indice = random.randint(0,len(solution)-1)
            solution[indice]=((solution[indice]+1) % 2)
            newBee=Bee(i,self.data,self.locIterations)
            newBee.solution = copy.deepcopy(solution)
            self.beeList.append(newBee)
            i+=1
        for bee in (self.beeList):
            lista=[j for j, n in enumerate(bee.solution) if n == 1]
            if (len(lista)== 0):
                bee.setSolution(bee.Rand(self.data.nb_attribs))
                
    def selectRefSol(self):
        self.beeList.sort(key=lambda Bee: Bee.reward, reverse=True)
        bestQuality=self.beeList[0].reward
        if(bestQuality>self.bestSolution.reward):
            self.bestSolution=self.beeList[0]
            self.nbChance=self.maxChance
            return self.bestSolution
        else:
            if(  (len(self.tabou)!=0) and  bestQuality > (self.tabou[len(self.tabou)-1].reward)  ):
                self.nbChance=self.maxChance
                return self.bestBeeQuality()
            else:
                self.nbChance-=1
                if(self.nbChance > 0): 
                    return self.bestBeeQuality()
                else :
                    return self.bestBeeDiversity()
    
    def distanceTabou(self,bee):
        distanceMin=self.data.nb_attribs
        for i in range(len(self.tabou)):
            cpt=0
            for j in range(self.data.nb_attribs):
                if (bee.solution[j] != self.tabou[i].solution[j]) :
                      cpt +=1
            if (cpt<=1) :
                return 0
            if (cpt < distanceMin) :
                distanceMin=cpt
        return distanceMin
    
    def bestBeeQuality(self):
        distance = 0
        i=0
        pos=-1
        while(i<self.nbrBees):
            max_val=self.beeList[i].reward
            nbUn=self.data.nbrUn(self.beeList[i].solution)
            while((i<self.nbrBees) and (self.data.evaluate(self.beeList[i].solution) == max_val)):
                distanceTemp=self.distanceTabou(self.beeList[i])
                nbUnTemp = self.data.nbrUn(self.beeList[i].solution)
                if(distanceTemp > distance) or ((distanceTemp == distance) and (nbUnTemp < nbUn)):
                    if((distanceTemp==distance) and (nbUnTemp<nbUn)):
                        print("On choisi la meilleure solution avec moins d'attributs")
                    nbUn=nbUnTemp
                    distance=distanceTemp
                    pos=i
                i+=1
            if(pos!=-1) :
                return self.beeList[pos]
        bee= Bee(-1,self.data,self.locIterations)
        bee.setSolution(bee.Rand(self.data.nb_attribs))
        return bee
            
    def bestBeeDiversity(self):
        max_val=0
        for i in range(len(self.beeList)):
            if (self.distanceTabou(self.beeList[i])> max_val) :
                max_val = self.distanceTabou(self.beeList[i])
        if (max_val==0):
            bee= Bee(-1,self.data,self.locIterations)
            bee.setSolution(bee.Rand(self.data.nb_attribs))
            return bee
        i=0
        while(i<len(self.beeList) and self.distanceTabou(self.beeList[i])!= max_val) :
            i+=1
        return self.beeList[i]
    
    def bso(self):
        i=0
        while(i<self.maxIterations):
            #print("La solution de référence est : ",self.refSolution.solution)
            self.tabou.append(self.refSolution)
            #print("Iteration N° : ",i)
            
            self.searchArea()
            """print("********************************************************")
            print("La zone de recherche est :")
            for k in range (self.nbrBees): 
                print(self.beeList[k].id)
                print(self.beeList[k].solution)
            print("*******************************************************")"""
            #La recherche locale
            
            for j in range(self.nbrBees):
                #self.beeList[j].localSearch()
                for episode in range(5):
                  self.beeList[j].ql_localSearch()
                #print( "Q-value of bee " + str(j) + " solution is : " + str(self.beeList[j].fitness))
            self.refSolution = self.selectRefSol()
            i+=1
        
        print("La meilleure solution trouvé est : ",self.bestSolution.solution)
        print("Nombre d'attributs utilisés : {0}".format(self.data.nbrUn(self.bestSolution.solution)))
        print("Son accuracy est : {0:.2f} ".format(self.bestSolution.reward*100))
        print("Sa Q-value est : ",self.bestSolution.fitness)

    
    def str_sol(self,mlist):
        result = ''
        for element in mlist:
            result += str(element)
        return result

      
class QLearning:
    def __init__(self,nb_atts,actions):
        self.actions = actions
        self.alpha = 0.1 # Facteur d'apprentissage
        self.gamma = 0.9
        self.epsilon = 0.1
        self.q_table = [ {} for i in range(nb_atts) ] #defaultdict(lambda : [0.0,0.0,0.0,0.0])

    def get_max_value(self,data,state,actions_vals):
        max_val = 0.0
        arg_max = 0
        for i in actions_vals:
            if self.get_q_value(data,state,i) >= max_val:
                max_val = self.get_q_value(data,state,i)
                arg_max = i
        if max_val == 0:
            arg_max = np.random.choice(actions_vals)
        return max_val,arg_max


    def get_q_value(self,data,state,action):
        if not self.str_sol(state) in self.q_table[self.nbrUn(state)]:
            self.q_table[self.nbrUn(state)][self.str_sol(state)] = {}

        if not str(action) in self.q_table[self.nbrUn(state)][self.str_sol(state)]:
            self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)] = data.evaluate(self.get_next_state(state,action))
            self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)] = 0
            
        return self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)]

    def set_q_value(self,state,action,val):
        self.q_table[self.nbrUn(state)][self.str_sol(state)][str(action)] = val

    def step(self,data,state):
        if np.random.uniform() > self.epsilon :
            #choisir la meilleure action
            action_values = self.actions
            argmax_actions=[] # La meilleure action peut ne pas exister donc on elle est choisie aléatoirement
            for ac in action_values :
                """if ac == self.get_max_value(state,action_values)[1]:
                    print("This action is to append : ", ac)
                    argmax_actions.append(ac)"""

                """if self.get_q_value(state,ac) == self.get_max_value(state,action_values)[1]:
                    print("This action is to append : ", ac)
                    argmax_actions.append(ac)"""

                #ac_state = self.get_next_state(state,ac)
                """print("Q-value for action :" + str(ac) + " is " + str(self.get_q_value(state,ac)))
                argmax_actions.append(ac)"""  


                ac_state_q_val = self.get_q_value(data,state,ac)
                if ( ac_state_q_val >= self.get_max_value(data,state,action_values)[0] ):
                    #print("Q-value for action :" + str(ac) + " is " + str(ac_state_q_val))
                    argmax_actions.append(ac)

            #print("This is argmax list : ",argmax_actions)
            if len(argmax_actions) != 0:
              next_action = np.random.choice(argmax_actions) 
            else:
              next_action = np.random.choice(action_values) 
            next_state = self.get_next_state(state,next_action)
            #print("The next state is :",next_state)
            
            #reward = data.evaluate(next_state)
        else :
            next_action = np.random.choice(self.actions)
            next_state = self.get_next_state(state,next_action)
            
            #reward = reward = data.evaluate(next_state)
            
        if self.epsilon > 0 :
            self.epsilon -= 0.0001 #Décrementer espsilon pour Arreter l'exploration aléatoire qu'on aura un politique optimale
        if self.epsilon < 0 :
            self.epsilon = 0

        return next_state, next_action #, reward


    def get_next_state(self,state,action):
        next_state = state.copy()
        next_state[action] = (next_state[action]+1) % 2
        return next_state
    
    def learn(self,data,current_state,current_action,reward,next_state):
        #print("current state : " + self.str_sol(current_state) + "| current action : " + str(current_action) + "| reward : "+ str(reward) + "| next state : "+ self.str_sol(next_state))
        
        next_action = self.step(data,next_state)[1] # step returns 3 values : next_state, next_action, and the reward
        new_q = reward + self.gamma * self.get_q_value(data,next_state,next_action)  #[0] is to pick q-value instead of [1] which is the accuracy of the new state 
        self.set_q_value(current_state,current_action,(1 - self.alpha)*self.get_q_value(data,current_state,current_action) + self.alpha*new_q)  

    #@staticmethod
    def str_sol(self,mlist):
        result = ''
        for element in mlist:
            result += str(element)
        return result

    def nbrUn(self,solution):
        return len([i for i, n in enumerate(solution) if n == 1])

      
      
class FsProblem :
    def __init__(self,data,qlearn):
        self.data=data
        self.nb_attribs= len(self.data.columns)-1 
        self.outPuts=self.data.iloc[:,self.nb_attribs]
        self.ql = qlearn
        self.nb_actions = len(self.ql.actions)
        self.classifier = KNeighborsClassifier(n_neighbors=5)

    def evaluate(self,solution):
        list=[i for i, n in enumerate(solution) if n == 1]
        if (len(list)== 0):
            return 0
         
        df = self.data.iloc[:,list]
        array=df.values
        nb_attribs =len(array[0])
        X = array[:,0:nb_attribs]
        Y = self.outPuts
        train_X, test_X, train_y, test_y = train_test_split(X, Y, 
                                                    random_state=0,
                                                    test_size=0.3
                                                    )
        classifier = KNeighborsClassifier(n_neighbors=5)
        classifier.fit(train_X,train_y)
        predict= classifier.predict(test_X) 
        return metrics.accuracy_score(predict,test_y)
    
    def evaluate2(self,solution):
        
        list=[i for i, n in enumerate(solution) if n == 1]
        if (len(list)== 0):
            return 0
        
        df = self.data.iloc[:,list]
        array=df.values
        nbrAttributs =len(array[0])
        X = array[:,0:nbrAttributs]
        Y = self.outPuts                                       
        classifier = KNeighborsClassifier(n_neighbors=5)
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
        results = cross_val_score(classifier, X, Y, cv= kfold, scoring='accuracy')
        
        return results.mean()
    
    def nbrUn(self,solution):
        return len([i for i, n in enumerate(solution) if n == 1])



class FSData:

    def __init__(self):
        #self.url = "https://raw.githubusercontent.com/Neofly4023/bso-fs/master/benchmarks/sonar.csv"
        self.url = "https://raw.githubusercontent.com/Neofly4023/bso-fs/master/benchmarks/glass.csv"
        self.dataset_name = "Glass"
        print("[START] Dataset reading")
        self.df = pd.read_csv(self.url,header=None)
        print("Shape : " + str(self.df.shape))
        print("[END] Dataset reading")
              #+ "\nDescription : \n")
        #self.df.describe()
        self.ql = QLearning(len(self.df.columns),self.attributs_to_flip(self.df))
        self.fsd = FsProblem(self.df,self.ql)
        
    
    def attributs_to_flip(self,dataset):

        return list(range(9))
    
    def run(self):
        t_init = time.time()
        for nbr_exec in range(1,7):
          print ("Execution N°{0}".format(str(nbr_exec)))
          self.ql = QLearning(len(self.df.columns),self.attributs_to_flip(self.df))
          self.fsd = FsProblem(self.df,self.ql)
          swarm = Swarm(self.fsd,5,3,10,5,5)
          t1 = time.time()
          swarm.bso()
          t2 = time.time()
          print("Time elapsed for execution N°{0} : {1:.2f} s\n".format(nbr_exec,t2-t1))
        t_end = time.time()
        print ("Execution time : {0:.2f}".format(t_end-t_init))
    
instance = FSData()
#sys.stdout = open('.\\logs\\file-'+ time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime()) +'.log','w')
classifier_name = str(type(instance.fsd.classifier)).strip('< > \' class ').split('.')[3]
#!mkdir logs
log_file = open('./logs/'+ time.strftime("%d-%m-%Y_%H-%M_", time.localtime()) + instance.dataset_name + '_' + classifier_name +'.txt','w+')
#sys.stdout = stream_tee(sys.stdout, log_file)
sys.stdout = log_file
instance.run()